In [1]:
from datasets import load_dataset
import datasets
from collections import Counter
import re
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import tokenizers

D:\conda_env\dl_learning20250612\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载 IMDB 数据集
cache_dir=r'.\data_file'
imdb_dataset = load_dataset("imdb",cache_dir=cache_dir)
# .map(): 对所有样本应用一个函数 (非常重要，用于预处理):

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at data_file\imdb\plain_text\0.0.0\e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Thu Jun  5 21:25:29 2025).


In [4]:
imdb_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [3]:
# 查看数据集结构
# 深拷贝复制数据
paragraphs=[i for i in imdb_dataset['unsupervised']['text']]

In [9]:
from transformers import AutoModel, AutoTokenizer

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
pretrainded_model = AutoModel.from_pretrained('bert-base-uncased')
pretrainded_model=pretrainded_model.to(device)

D:\conda_env\dl_learning20250612\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zzz\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
pretrainded_model.embeddings.word_embeddings=nn.Embedding(20000,768,padding_idx=0)

In [20]:
pretrainded_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(20000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False